In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import cufflinks as cf
cf.go_offline(connected=True)
from tqdm import tqdm
import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
import pymysql
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
import warnings

In [2]:
warnings.filterwarnings(action='ignore')

## 구축한 머신러닝 데이터 불러오기

In [3]:
def load_file(TYPE,trading, days,target):
    with open('../data/{}_{}_{}_{}.pickle'.format(TYPE,trading, days,target), 'rb') as f:
        data = pickle.load(f)
    return data

In [4]:
def dump_model(model,trading, days,target):
    with open('../model/{}_{}_{}_{}.pickle'.format(model,trading, days,target), 'wb') as f:
        pickle.dump(model, f)

In [5]:
def load_model(model,trading, days,target):
    with open('../model/{}_{}_{}_{}.pickle'.format(model,trading, days,target), 'rb') as f:
        model = pickle.load(f)
    return model

In [6]:
train = load_file('train',1000000000000,10,0.02)
test = load_file('test',1000000000000,10,0.02)

### train

In [7]:
lst_code2date = []
trainX = []
trainY = []

for line in train:
    code, date, x, y = line
    lst_code2date.append([code,date])
    trainX.append(list(map(int, x.split(','))))
    trainY.append(int(y))

In [8]:
trainX = np.array(trainX)
trainY = np.array(trainY)

## test

In [9]:
lst_code2date = []
testX = []
testY = []

for line in test:
    code, date, x, y = line
    lst_code2date.append([code,date])
    testX.append(list(map(int, x.split(','))))
    testY.append(int(y))
testX = np.array(testX)
testY = np.array(testY)

### Modeling

In [10]:
LR = LogisticRegression(random_state = 42)
LR.fit(trainX, trainY)

LogisticRegression(random_state=42)

In [11]:
dump_model(LR,1000000000000,10,0.02)

In [12]:
LR = load_model(LogisticRegression(random_state = 42),1000000000000,10,0.02)

In [13]:
predY = LR.predict_proba(testX) # predict_proba 함수는 예측한 값을 확률 값으로 출력
predY2 = LR.predict(testX) # predict 함수는 예측한 값을 이진 값(1 또는 0)으로 출력

In [21]:
predY

array([[0.47270631, 0.52729369],
       [0.48977521, 0.51022479],
       [0.50975051, 0.49024949],
       [0.49153171, 0.50846829],
       [0.41687951, 0.58312049],
       [0.64937167, 0.35062833],
       [0.43121972, 0.56878028],
       [0.49671836, 0.50328164],
       [0.43118093, 0.56881907],
       [0.57162541, 0.42837459],
       [0.45877259, 0.54122741],
       [0.45645762, 0.54354238],
       [0.5212776 , 0.4787224 ],
       [0.5562763 , 0.4437237 ],
       [0.70471271, 0.29528729],
       [0.42362539, 0.57637461],
       [0.56194577, 0.43805423],
       [0.47800619, 0.52199381],
       [0.54822839, 0.45177161],
       [0.52735642, 0.47264358],
       [0.53410522, 0.46589478],
       [0.47923608, 0.52076392],
       [0.51399554, 0.48600446],
       [0.48900469, 0.51099531],
       [0.47057139, 0.52942861],
       [0.46195777, 0.53804223],
       [0.49344916, 0.50655084],
       [0.47935649, 0.52064351],
       [0.46373406, 0.53626594],
       [0.58107086, 0.41892914],
       [0.

In [15]:
predY2

array([1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1])

## 주문 요청 일지

In [16]:
lst_output = []

for (code,date),y in zip(lst_code2date, predY):
    if y[1] >= 0.7:
        lst_output.append([code, date, 'buy','r50'])
        lst_output.append([code, date+"n", 'sell', 'all'])
    elif y[1] >= 0.5:
        lst_output.append([code, date, 'buy', 'r30'])
        lst_output.append([code, date+"n", 'sell', 'all'])
    elif y[1] >= 0.3:
        lst_output.append([code, date, 'buy', 'r10'])
        lst_output.append([code, date+"n", 'sell', 'all'])

In [17]:
lst_output.sort(key = lambda x: x[1]) # date 기준으로 주문 요청 결과 정렬

OF = open('../data/order_request.txt','w')
for row in lst_output:
    OF.write('\t'.join(map(str, row)) + '\n')
OF.close()

In [18]:
# database
con = pymysql.connect(

)

cursor = con.cursor()

## 수익률 계산

In [19]:
start_money = 10000000 # 초기 현금 1천만원
money = start_money
dic_code2num ={}  # 보유 종목


IF = open('../data/order_request.txt','r')
for i, line in tqdm(enumerate(IF)): #주문 일지를 한 줄 읽어 옴
    code, date, request, amount = line.strip().split("\t")
    
    ##############################################################################################
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2021-01-01' AND '2021-07-01'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = con)
    lst_stock = stock.values.tolist()
    
    for idx, row in enumerate(lst_stock):
        Date = row[0].strftime('%Y%m%d')        
        if date.endswith('n'):
            if date.rstrip('n') == Date:
                sell_close = lst_stock[idx+1][4]
        elif date == Date:
            buy_close = lst_stock[idx][4]

    ##############################################################################################
    
    if request == 'buy': # buy인 경우
        if amount.startswith('r'):
            request_money = money * float(amount.lstrip("r")) / 100
        elif amount == 'all':
            request_money = money
        elif amount.isdigit():
            request_money = int(amount)
        # elif amount == ~~~~~    ##### 기타 필요한 매수 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')
        request_money = min(request_money, money)
        buy_num = int(request_money / buy_close)
        money -= buy_num * buy_close  # 현재 금액(money)을 실제 매수액을 뺀 만큼 업데이트
        if code not in dic_code2num:
            dic_code2num[code] = 0
        dic_code2num[code] += buy_num # 보유 종목 데이터에 구매 종목(code)를 매수 개수 만큼 증가
    if request == 'sell': # sell인 경우
        if amount == 'all':
            sell_num = dic_code2num[code]
        # elif amount == ~~~~~    ##### 기타 필요한 매도 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')            
        money += sell_num * sell_close
        dic_code2num[code] -= sell_num
        if dic_code2num[code] == 0:
            del dic_code2num[code]
IF.close()            
            
if dic_code2num != {}: # 매매가 종료되었는데 보유 종목이 있으면
    raise Exception('Not empty stock') 

print("Final earning rate : {} %".format(str((money-start_money) / start_money * 100)))

372it [00:04, 89.42it/s]

Final earning rate : -28.06787 %
